# EECE7150 AFR, Homework 1

In [20]:
import numpy as np
from enum import Enum

### 2(b) Simulator to generate random weather sequences

 #### (i) How you initialize the simulation: 
 
 The simulation is initialized with an initial state (sunny, cloudy or rainy) and then based on this information, the next day's state (weather) is sampled using the transition matrix


#### (ii) How many transitions you consider before taking a result:

 Ideally, we asymptotically reach the stationary probabilities, but in simulations atleast about 100 transitions are needed before taking a result



#### (iii) How many simulation runs you conducted to arrive at the statistics for the stationary distribution:
 
 I tested this empirically. I ran 5, 10, 100, 1000, 10000 simulations and found out that after about 1000 iterations, the static distribution converges



In [21]:
class Weather(Enum):
    SUNNY  = 0
    CLOUDY = 1
    RAINY  = 2

class WeatherSim():
    def __init__(self) -> None:
        self.transition_mat = np.array([[.8, .2, .0],
                                        [.4, .4, .2],
                                        [.2, .6, .2]])
        
        self.weather_sequence = None
        
    def simulate(self, today: Weather, iterations:int) -> Weather:
        weather_seq = np.array([today.value])
        for _ in range(1, iterations):
            tomorrow:Weather = np.random.choice([Weather.SUNNY,
                                                 Weather.CLOUDY,
                                                 Weather.RAINY], 
                                                 p = self.transition_mat[today.value,:])
            today = tomorrow
            weather_seq = np.append(weather_seq, tomorrow.value)

        self.weather_sequence = weather_seq
        self.__get_static_prob(iterations)

    def __get_static_prob(self, iterations:int):
        stat_prob = np.unique(self.weather_sequence, return_counts=True)[1]/(iterations+1)
        print("Static Probability:")
        print(f"{Weather.SUNNY.name}  = {stat_prob[Weather.SUNNY.value]}")
        print(f"{Weather.CLOUDY.name} = {stat_prob[Weather.CLOUDY.value]}")
        print(f"{Weather.RAINY.name}  = {stat_prob[Weather.RAINY.value]}")

### 2(c) Stationary distribution based on the transition matrix

In [22]:
simulator = WeatherSim()
today = Weather.RAINY
simulator.simulate(today, 10000)

Static Probability:
SUNNY  = 0.6497350264973503
CLOUDY = 0.2813718628137186
RAINY  = 0.06879312068793121


### 2(d) Closed form solution

Essentially, if $p$ represents the static probability distribution for each state (weather), and $A$ represents the state transition matrix, then $$p = A^{T}p$$

So technically $p$ is the eigenvector corresponding to the eigenvalue 1. This is shown below (The same effect is achieved by diagonalization of A, but I find this method more intuitive)

In [23]:
e_values, e_vectors = np.linalg.eig(simulator.transition_mat.T)
stat_distribution = e_vectors[:,0]/np.sum(e_vectors[:,0])

print(f"Eigen Values:\n{e_values[0]}\n{e_values[1]}\n{e_values[2]}\n")
print(f"Normalized eigen vector corresponding to the eigenvalue 1:\n{stat_distribution}")

Eigen Values:
1.0000000000000002
0.48284271247461874
-0.08284271247461897

Normalized eigen vector corresponding to the eigenvalue 1:
[0.64285714 0.28571429 0.07142857]


The normalized eigen vector corresponding to eigen value 1 is the stationary distribution and matches with the distribution found via simulation